In [ ]:
# precisa ter o tesnroflow versao 2.10 no máximo pq tiraram suporte de GPU no windows por algum motivo
#!conda install -c conda-forge cudatoolkit=11.2 cudnn=8.1.0
#!pip install --upgrade pip
#!pip install "tensorflow<2.11"
!pip list

In [1]:
import datetime as dt
import wget
import os
import pandas as pd
import numpy as np
from zipfile import ZipFile
from timeit import default_timer as timer
from pyts.image import GramianAngularField
from pyts.image import MarkovTransitionField
from keras.utils import load_img
from keras.utils import save_img
from keras.utils import img_to_array
from keras.utils import array_to_img
from keras.regularizers import l2
import funcoes as f
import asyncio
#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1' # or any {'0', '1', '2'}
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image_dataset_from_directory
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
import plotly.express as px
from sklearn.ensemble import BaggingClassifier
from sklearn.multiclass import OneVsRestClassifier
from joblib import dump, load
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import absl.logging
absl.logging.set_verbosity(absl.logging.ERROR)
tf.get_logger().setLevel('WARNING')
tf.keras.mixed_precision.set_global_policy("mixed_float16")
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [2]:
# define o ticker do ativo que vou usar pra treinar o modelo
ticker = "BTCUSDT"
# timeframes precisa estar em ordem crescente e começar em 1s ou 1m
timeframes = ("1m", "5m", "15m", "30m", "1h", "2h", "4h", "8h", "1d")
# quantos períodos vamos olhar pro passado
lookback = 20
# numero de quantis pra usar no markov transition field. Precisa tunar.
quantis = 3
# pega número de timeframes por minuto ou segundo
timeframes_padronizado = tuple(f.timeframes_mesma_unidade(timeframes))
# tamanho das batches de treinamento. Tem mais a ver com velocidade de processamento e não interfere muito nos resultados
batch_size = 1024#512
# set seed pros resultados não variarem
seed = 777
tf.keras.utils.set_random_seed(seed)
np.random.seed(seed)
# pega tamanho da imagem (input)
img_shape = f.image_shape()
# porcentagem dos dados que vou usar pra treino e teste
pct_imagens_teste = 0.1
# numero de modelos pro ensemble
n_modelos=5

# Prepara os dados

### Baixa e corrige dados

In [ ]:
# baixa os dados e junta cada timeframe e junta em 1 csv por timeframe
# retorna None se arquivo processado já existe e timeframe se ele criou um arquivo do 0
lista_processados = []
for timeframe in timeframes:
    lista_processados.append(f.baixa_e_concatena(ticker= ticker, timeframe=timeframe, ano_inicial=2017))

In [ ]:
# insere linhas sem dados e preenche com 0
# só corrige os timeframes que foram processados anteriormente
lista_p_correcao = [item for item in lista_processados if item != None]
f.corrige_arquivos(lista_p_correcao)

### Cria janelas, GAF's e salva imagens
Imagens ficam localizadas na pasta dados e são separadas por clissificação de compra ou venda

In [ ]:
# cria uma lista de numpy arrays com os dados de cada timeframe.
# Eles estão organizados na mesma ordem que os timeframes
dfs_close = [(pd.read_csv(f"./Dados/Processados/BTCUSDT-{timeframe}.csv", 
                      usecols=["Close time", "Close"])[::-1][["Close", "Close time"]]).to_numpy(dtype="float32") for timeframe in timeframes]
dfs_close = tuple(dfs_close)

# faz a mesma coisa pros dados de volume
dfs_volume = [(pd.read_csv(f"./Dados/Processados/BTCUSDT-{timeframe}.csv", 
                      usecols=["Close time", "Volume"])[::-1][["Volume", "Close time"]]).to_numpy(dtype="float32") for timeframe in timeframes]
dfs_volume = tuple(dfs_volume)

dfs_close[0], dfs_volume[0]

In [ ]:
if __name__ == '__main__':
    ultima_linha = len(dfs_close[0])-(lookback*timeframes_padronizado[-1])
    print("qtd de imagens para treino", ultima_linha-int(ultima_linha*pct_imagens_teste))
    print("qtd de imagens para teste", int(ultima_linha*pct_imagens_teste))
    # linhas de teste são as primeiras x%, pois os dados mais recentes vem antes
    # linhas de treino são as restantes
    linhas_teste = range(0, int(ultima_linha*pct_imagens_teste))
    linhas_treino = range(int(ultima_linha*pct_imagens_teste)+1, ultima_linha+1)
    
    # cria imagens usando multithreading
    f.roda_paralelo(linhas_treino, linhas_teste, dfs_close, dfs_volume, lookback, quantis)

# Modelos

In [12]:
def CNN_hinge(img_shape=img_shape):
    # define our MLP network
    model = keras.Sequential()
    model.add(layers.Input(shape=img_shape))
    
    # reescala os dados pra entre 0 e 1
    model.add(layers.Rescaling(1./255))
    
    # primeira convolucao
    model.add(layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))
    model.add(layers.MaxPool2D(pool_size=(1, 2)))
    model.add(layers.Dropout(0.15))
    
    model.add(layers.Flatten(name="flatten"))
    model.add(layers.Dense(16, activation="relu"))
    model.add(layers.Dense(8, activation="relu"))
    model.add(layers.Dense(1, kernel_regularizer=l2(0.01))) # aqui mandam usar l2 0,01
    model.add(layers.Activation('linear')) # aqui mandam por linear
    #model.add(Activation('tanh')) # tentar com tanh 
    
    model.compile(loss = "categorical_hinge", optimizer="Adam", metrics=["accuracy",
                                                                     keras.metrics.Precision(),
                                                                     keras.metrics.Recall(), 
                                                                     keras.metrics.AUC()])
    return model

In [5]:
# rede CNN padrão
def CNN(img_shape=img_shape):
    model = keras.Sequential()
    model.add(layers.Input(shape=img_shape))
    
    # reescala os dados pra entre 0 e 1
    model.add(layers.Rescaling(1./255))
    
    # convolucao
    model.add(layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))
    model.add(layers.MaxPool2D(pool_size=(1, 2)))
    model.add(layers.Dropout(0.15))
    
    # classificação
    model.add(layers.Flatten(name="flatten"))
    model.add(layers.Dense(16, activation="linear"))
    model.add(layers.Dense(8, activation="linear"))
    model.add(layers.Dense(1, activation="sigmoid"))
    

    return model

# Treino

Pega dados

In [4]:
train_data, validation_data = image_dataset_from_directory("./Dados/Imagens/treino/",
                                                   label_mode="binary",
                                                  image_size=img_shape[:2],
                                                  batch_size=batch_size,
                                                  subset="both",
                                                  validation_split=0.1,
                                                  seed=seed)

Found 2284487 files belonging to 2 classes.
Using 2056039 files for training.
Using 228448 files for validation.


Treina n modelos

In [5]:
modelos = [CNN_hinge() for i in range(n_modelos)]
resultado_CNN_hinge = np.empty((n_modelos, 5))
i = 0
for model in modelos:
    print("Modelo:", i)
    # precisa definir o checkpoint antes de começar cada CNN. Se não acaba usando um do outro
    checkpoint = ModelCheckpoint(f"modelo {i}", monitor='val_accuracy', verbose=0, save_best_only=True, mode='max')
    # para de treinar se a acuracia de treino parar de aumentar por 3 epochs
    es = EarlyStopping(monitor='val_accuracy', patience=4)
    callbacks_list = [checkpoint, es]

    # treina CNN hinge
    model.fit(train_data,  epochs=10, use_multiprocessing=True, callbacks=callbacks_list, validation_data=validation_data) #verbose=0
    model = keras.models.load_model(f"modelo {i}")
    resultado_CNN_hinge[i] = model.evaluate(validation_data)
    i = i + 1

Modelo: 0
Epoch 1/10
2008/2008 [==============================] - 342s 168ms/step - loss: 1.0266 - accuracy: 0.5047 - precision: 0.5122 - recall: 0.1618 - auc: 0.5054 - val_loss: 0.9886 - val_accuracy: 0.5110 - val_precision: 0.5205 - val_recall: 0.2633 - val_auc: 0.5147
Epoch 2/10
2008/2008 [==============================] - 361s 179ms/step - loss: 0.9709 - accuracy: 0.5195 - precision: 0.5287 - recall: 0.3434 - auc: 0.5241 - val_loss: 0.9655 - val_accuracy: 0.5171 - val_precision: 0.5451 - val_recall: 0.1989 - val_auc: 0.5224
Epoch 3/10
2008/2008 [==============================] - 369s 183ms/step - loss: 0.9487 - accuracy: 0.5290 - precision: 0.5341 - recall: 0.4418 - auc: 0.5349 - val_loss: 0.9543 - val_accuracy: 0.5226 - val_precision: 0.5413 - val_recall: 0.2878 - val_auc: 0.5276
Epoch 4/10
2008/2008 [==============================] - 372s 185ms/step - loss: 0.9405 - accuracy: 0.5322 - precision: 0.5368 - recall: 0.4582 - auc: 0.5391 - val_loss: 0.9671 - val_accuracy: 0.5163 - val

In [ ]:
# faz votacao dos resultados. depois repete pro CNN normal

# Teste

In [22]:
test_data = image_dataset_from_directory("./Dados/Imagens/teste/",
                                                   label_mode="binary",
                                                 image_size=img_shape[:2],
                                                  batch_size=batch_size,
                                                 shuffle=False,
                                                  seed=seed)

Found 279533 files belonging to 2 classes.


In [23]:
# mostra o resultado pra cada um dos modelos treinados
resultado_CNN_hinge = np.empty((n_modelos, 5))
i = 0
for i in range(n_modelos):
    model = keras.models.load_model(f"modelo {i}")
    print(f"modelo {i}")
    resultado_CNN_hinge[i] = model.evaluate(test_data)
# transforma resultados CNN em df e tira a loss
resultado_CNN_hinge = pd.DataFrame(resultado_CNN_hinge, columns=["loss", "accuracy", "precision", "recall", "AUC"]).drop(columns=["loss"])
# calcula f1
resultado_CNN_hinge["f1-score"] = 2*(resultado_CNN_hinge["precision"]*resultado_CNN_hinge["recall"])/(resultado_CNN_hinge["precision"]+resultado_CNN_hinge["recall"])
# poe 0 nos nans
resultado_CNN_hinge.fillna(0, inplace=True)
# reordena as colunas
colunas = ["accuracy", "precision", "recall", "f1-score", "AUC"]
resultado_CNN_hinge = resultado_CNN_hinge[colunas]
display(resultado_CNN_hinge)

modelo 0
273/273 [==============================] - 43s 155ms/step - loss: 0.9494 - accuracy: 0.5281 - precision: 0.5285 - recall: 0.5214 - auc: 0.5325
modelo 1
273/273 [==============================] - 38s 140ms/step - loss: 1.0024 - accuracy: 0.4999 - precision_1: 0.0000e+00 - recall_1: 0.0000e+00 - auc_1: 0.5000
modelo 2
273/273 [==============================] - 37s 133ms/step - loss: 0.9500 - accuracy: 0.5274 - precision_2: 0.5336 - recall_2: 0.4368 - auc_2: 0.5323
modelo 3
273/273 [==============================] - 37s 135ms/step - loss: 0.9503 - accuracy: 0.5287 - precision_3: 0.5272 - recall_3: 0.5574 - auc_3: 0.5334
modelo 4
273/273 [==============================] - 36s 132ms/step - loss: 1.0026 - accuracy: 0.4999 - precision_4: 0.0000e+00 - recall_4: 0.0000e+00 - auc_4: 0.5000


,accuracy,precision,recall,f1-score,AUC
0,0.528052,0.528483,0.521401,0.524918,0.532454
1,0.499948,0.000000,0.000000,0.000000,0.500000
2,0.527426,0.533558,0.436848,0.480384,0.532275
3,0.528714,0.527207,0.557358,0.541863,0.533412
4,0.499948,0.000000,0.000000,0.000000,0.500000


In [ ]:
# votacao

In [24]:
# pega labels reais
y_true = np.concatenate([y for x, y in test_data], axis=0)
# converte y_true pra range entre -1 e 1, que é o mesmo das predições. 
#Daria tbm pra fazer o inverso, transformando pra 0 e 1: y_pred = (y_pred+1)/2
#y_true = y_true*2-1
y_true

array([[0.],
       [0.],
       [0.],
       ...,
       [1.],
       [1.],
       [1.]], dtype=float32)

In [25]:
# pega predições
y_pred = pd.DataFrame()
i = 0
for i in range(n_modelos):
    model = keras.models.load_model(f"modelo {i}")
    print(f"modelo {i}")
    y_pred[f"modelo {i}"] = model.predict(test_data).ravel().tolist()
y_pred

modelo 0
273/273 [==============================] - 36s 132ms/step
modelo 1
273/273 [==============================] - 37s 134ms/step
modelo 2
273/273 [==============================] - 39s 141ms/step
modelo 3
273/273 [==============================] - 36s 133ms/step
modelo 4
273/273 [==============================] - 36s 131ms/step


,modelo 0,modelo 1,modelo 2,modelo 3,modelo 4
0,-0.917480,-0.072083,-0.931152,-0.929199,-0.096252
1,0.851074,-0.072083,-0.670898,0.881836,-0.096252
2,-0.861816,-0.072083,-0.904297,-0.908691,-0.096252
3,0.901855,-0.072083,0.842773,0.948242,-0.096252
4,0.876465,-0.072083,0.515625,0.972168,-0.096252
...,...,...,...,...,...
279528,-0.898438,-0.072083,-0.916504,-0.027939,-0.096252
279529,0.896484,-0.072083,-0.428955,0.973145,-0.096252
279530,0.876953,-0.072083,-0.058716,0.951660,-0.096252
279531,0.905762,-0.072083,0.833984,0.951172,-0.096252


In [84]:
# transforma predicoes em -1, 0 ou 1 para votação
def decisao(x, threshold):
    '''
    Transforma valores em 1, -1 ou 0 com base no valor do threshold.
    
    x: valor entre -1 e 1 que iremos transformar em -1, 0 ou 1
    
    threshold: valor entre 0.5 e 1 que irá definir a % de certeza 
    para que x seja classificado entre -1 (long), 1 (short) ou 0 (nada)
    '''
    threshold = (threshold*2)-1
    if (x >= threshold):
        x = 1
    elif (x <= (0-threshold)):
        x = -1
    else:
        x = 0
    return x

In [85]:
# decide qual será voto com base no threshold
threshold = 0.55
y_pred_ensemble = y_pred.applymap(lambda x: decisao(x, threshold))
# calcula a moda por linha para definir qual o consenso do modelo
y_pred_ensemble = y_pred_ensemble.mode(axis=1)
# se tem 2 modas ou mais, classificação deve ser 0 (não fazer nada)
if len(y_pred_ensemble.columns) >= 2:
    y_pred_ensemble.loc[y_pred_ensemble.iloc[:, 1].notna(), y_pred_ensemble.columns[0]] = 0
    y_pred_ensemble.drop(columns=y_pred_ensemble.columns[1], inplace=True)
y_pred_ensemble.columns = ["moda"]
y_pred_ensemble

,moda
0,-1.0
1,0.0
2,-1.0
3,1.0
4,1.0
...,...
279528,0.0
279529,0.0
279530,0.0
279531,1.0


In [ ]:
# calcular metricas pro y_pred_ensemble

# brincadeiras

In [23]:
model = keras.models.load_model(f"modelo 0")

# pega predicoes
y_pred = model.predict(test_data)
y_pred = (y_pred+1)/2
# faz ROC curve
import plotly.express as px
from sklearn.metrics import roc_curve, auc


fpr, tpr, thresholds = roc_curve(y_true, y_pred,)
fig = px.area(
    x=fpr, y=tpr,
    title=f'ROC Curve (AUC={auc(fpr, tpr):.4f})',
    labels=dict(x='False Positive Rate', y='True Positive Rate'),
    width=700, height=500
)
fig.add_shape(
    type='line', line=dict(dash='dash'),
    x0=0, x1=1, y0=0, y1=1
)

fig.update_yaxes(scaleanchor="x", scaleratio=1)
fig.update_xaxes(constrain='domain')
fig.show()

# get the best threshold
J = tpr - fpr
ix = J.argmax()
best_thresh = thresholds[ix]
print('Best Threshold=%f' % (best_thresh))

273/273 [==============================] - 38s 137ms/step


Best Threshold=0.609863


In [ ]:
# Predict labels with models
labels = []
for m in models:
    predicts = np.argmax(m.predict(test), axis=1)
    labels.append(predicts)
    
# Ensemble with voting
labels = np.array(labels)
labels = np.transpose(labels, (1, 0))
labels = scipy.stats.mode(labels, axis=-1)[0]
labels = np.squeeze(labels)